In [ ]:
from langchain_community.document_loaders import TextLoader
loader=TextLoader("youtube_transcript.txt")
text_doc=loader.load()
text_doc

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from bs4 import SoupStrainer

loader = WebBaseLoader(
    web_path="https://www.aljazeera.com/news/liveblog/2025/6/20/live-iran-israel-continue-missile-fire-irans-fm-to-meet-eu-counterparts",
    bs_kwargs=dict(parse_only=SoupStrainer(class_="article-header"))
)

k = loader.load()
k

In [11]:
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader("report (1).pdf")
pdf_doc=loader.load()
pdf_doc

[Document(metadata={'producer': '3.0.24 (5.1.10)', 'creator': 'PDFCreator Online (www.pdfforge.org/online)', 'creationdate': '2025-06-15T05:43:00+00:00', 'title': 'Merged with PDFCreator Online', 'moddate': '2025-06-16T05:39:30+02:00', 'source': 'report (1).pdf', 'total_pages': 79, 'page': 0, 'page_label': '1'}, page_content='OpenHands: Complete\nTechnical Guide for Beginners\n \n1. Project Overview\n2. Core Concepts\n3. Architecture Overview\n4. Repository Structure\n5. Technical Stack\n6. Key Components Deep Dive\n7. Development Workflow\n8. Agent System\n9. Runtime Environments\n10. Frontend Application\n11. Evaluation Framework\n12. Microagents System\n13. Configuration and Setup\n14. Testing Strategy\n15. Deployment Options\n16. Contributing Guidelines\n17. Advanced Topics\n18. Performance Optimization\n19. Security Considerations\n20. Troubleshooting Guide\n21. API Reference\n22. Extension Development\nProject Overview\nWhat is OpenHands?\nOpenHands (formerly OpenDevin) is a revo

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
split=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
text=split.split_documents(pdf_doc)

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
embed=OllamaEmbeddings(model="llama3.2:latest")
db_f=FAISS.from_documents(text[0:20], embedding=embed)


In [35]:
result=db_f.similarity_search("what is main topic")

In [15]:
from langchain_core.prompts import ChatPromptTemplate
template=ChatPromptTemplate.from_template(
    """you are an intelligent AI and u will answer the questions based on the context provided
     and u are in a competetiton with a similar AI and if the user finds the other ai answer more useful it will stop using u
       be very precise and informative in your answer and maintain a high level of accuracy and be a friendly talkative AI.
       <context>
       {context}
       </context>
       question:{input}"""
)

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_groq import ChatGroq
import os
os.environ["GROQ_API_KEY"]="your api key"
model1=ChatGroq(model="meta-llama/llama-4-scout-17b-16e-instruct",temperature=0.2)
chain1=create_stuff_documents_chain(model1,template)
response = chain1.invoke({
    "input": "what is the maintopic of the report?",
    "context": result 
})
print(response)

I'm excited to help and provide you with a precise and informative answer.

Based on the context provided, the main topic of the report appears to be **OpenHands: An AI Development Platform**. Specifically, it seems to be an overview of the platform's architecture, principles, and features, which include Pattern Recognition, Automated Refactoring, Emerging Technology Adoption, and more.

The report highlights OpenHands' technical philosophy, AI safety and ethics, performance and scalability, and core concepts such as agents, autonomous decision-making, and collaboration.

I'm glad I could provide a clear and concise answer! If you have more questions, feel free to ask, and I'll do my best to help. 

**By the way, I hope my answer is more useful than my competitor's, and I'm looking forward to helping you further!**


In [37]:
retrieve=db_f.as_retriever
retrieve

<bound method VectorStore.as_retriever of <langchain_community.vectorstores.faiss.FAISS object at 0x0000018D5F2BFBF0>>

In [40]:
from langchain.chains import create_retrieval_chain
retriever = db_f.as_retriever()
chain2 = create_retrieval_chain(retriever, chain1)

# Run the retrieval chain with a query
response = chain2.invoke({"input": "what is the maintopic of the report?"})
print(response)

{'input': 'what is the maintopic of the report?', 'context': [Document(id='fbdc259e-9e1b-4619-be83-5aab3e9cd138', metadata={'producer': '3.0.24 (5.1.10)', 'creator': 'PDFCreator Online (www.pdfforge.org/online)', 'creationdate': '2025-06-15T05:43:00+00:00', 'title': 'Merged with PDFCreator Online', 'moddate': '2025-06-16T05:39:30+02:00', 'source': 'report (1).pdf', 'total_pages': 79, 'page': 4, 'page_label': '5'}, page_content='each with unique skills and capabilities.\nAgent Characteristics\nAutonomous Decision Making: Agents can analyze situations and choose appropriate actions without constant human\nguidance\nGoal-Oriented Behavior: Each agent works towards completing specific objectives\nLearning Capability: Agents improve their performance based on feedback and experience\nSpecialization: Different agents excel in different domains (coding, web browsing, testing, etc.)'), Document(id='847a8a64-e0a1-4d64-9c6e-83413fa42f13', metadata={'producer': '3.0.24 (5.1.10)', 'creator': 'PDFC